# Prototype Function for Pricing a Gas Storage Contract

In [1]:
def price_storage_contract(
    injection_dates, withdrawal_dates,
    purchase_prices, sale_prices,
    injection_rate, withdrawal_rate,
    max_storage, storage_cost_per_month,
    injection_cost_per_mmbtu, withdrawal_cost_per_mmbtu,
    transport_cost_per_movement
):
    """
    Function to price a natural gas storage contract.

    Parameters:
    - injection_dates (list of str): Dates when gas is injected.
    - withdrawal_dates (list of str): Dates when gas is withdrawn.
    - purchase_prices (dict): {date: price} mapping for purchase.
    - sale_prices (dict): {date: price} mapping for sale.
    - injection_rate (float): Maximum gas that can be injected per date.
    - withdrawal_rate (float): Maximum gas that can be withdrawn per date.
    - max_storage (float): Maximum storage capacity in MMBtu.
    - storage_cost_per_month (float): Monthly storage cost.
    - injection_cost_per_mmbtu (float): Cost per MMBtu for injection.
    - withdrawal_cost_per_mmbtu (float): Cost per MMBtu for withdrawal.
    - transport_cost_per_movement (float): Fixed cost per transport event.

    Returns:
    - contract_value (float): Net value of the contract.
    """

    import pandas as pd

    # Convert dates to datetime
    injection_dates = pd.to_datetime(injection_dates)
    withdrawal_dates = pd.to_datetime(withdrawal_dates)

    # Step 1: Compute injected volume
    total_injected_volume = 0
    for date in injection_dates:
        price = purchase_prices.get(str(date.date()), None)
        if price is None:
            raise ValueError(f"Missing purchase price for {date.date()}")
        inject_amount = min(injection_rate, max_storage - total_injected_volume)
        total_injected_volume += inject_amount

    # Step 2: Compute withdrawn volume
    total_withdrawn_volume = 0
    for date in withdrawal_dates:
        price = sale_prices.get(str(date.date()), None)
        if price is None:
            raise ValueError(f"Missing sale price for {date.date()}")
        withdraw_amount = min(withdrawal_rate, total_injected_volume - total_withdrawn_volume)
        total_withdrawn_volume += withdraw_amount

    # Step 3: Compute revenues
    total_revenue = sum(sale_prices[str(date.date())] * min(withdrawal_rate, total_withdrawn_volume)
                        for date in withdrawal_dates)

    # Step 4: Compute costs
    total_cost = sum(purchase_prices[str(date.date())] * min(injection_rate, max_storage)
                     for date in injection_dates)

    storage_duration = (withdrawal_dates[-1] - injection_dates[0]).days // 30  # Convert days to months
    storage_cost = storage_duration * storage_cost_per_month

    injection_cost = total_injected_volume * injection_cost_per_mmbtu
    withdrawal_cost = total_withdrawn_volume * withdrawal_cost_per_mmbtu
    transport_cost = 2 * transport_cost_per_movement  # One for injection, one for withdrawal

    total_expenses = total_cost + storage_cost + injection_cost + withdrawal_cost + transport_cost

    # Compute final contract value
    contract_value = total_revenue - total_expenses

    return contract_value


# Test the Model with Sample Inputs

In [2]:
# Sample Inputs
injection_dates = ["2024-06-01", "2024-06-15"]
withdrawal_dates = ["2024-12-01", "2024-12-15"]

purchase_prices = {
    "2024-06-01": 2.5,
    "2024-06-15": 2.6
}

sale_prices = {
    "2024-12-01": 3.8,
    "2024-12-15": 3.7
}

# Contract parameters
injection_rate = 500000  # MMBtu per transaction
withdrawal_rate = 500000  # MMBtu per transaction
max_storage = 1000000  # MMBtu
storage_cost_per_month = 100000  # $ per month
injection_cost_per_mmbtu = 10000 / 1e6  # $10K per 1M MMBtu
withdrawal_cost_per_mmbtu = 10000 / 1e6  # $10K per 1M MMBtu
transport_cost_per_movement = 50000  # $ per transport event

# Run Model
contract_value = price_storage_contract(
    injection_dates, withdrawal_dates,
    purchase_prices, sale_prices,
    injection_rate, withdrawal_rate,
    max_storage, storage_cost_per_month,
    injection_cost_per_mmbtu, withdrawal_cost_per_mmbtu,
    transport_cost_per_movement
)

print(f"🔹 Estimated Contract Value: ${contract_value:,.2f}")


🔹 Estimated Contract Value: $480,000.00
